# Linear Covariance Testing

Use linear mixed effect models to test the effect of the guide on the covariance of IRF4 and BATF and target genes.

### Import

In [1]:
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [2]:
repo_path = '/Users/mincheolkim/Github/'
data_path = '/Users/mincheolkim/Documents/'

In [3]:
import sys
sys.path.append(repo_path + 'scVI')
sys.path.append(repo_path + 'scVI-extensions')

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import imp
import numpy as np
import pandas as pd

In [5]:
import scvi_extensions.dataset.supervised_data_loader as sdl
import scvi_extensions.dataset.cropseq as cs
import scvi_extensions.inference.supervised_variational_inference as svi
import scvi_extensions.hypothesis_testing.mean as mn
import scvi_extensions.hypothesis_testing.variance as vr
import scvi_extensions.dataset.label_data_loader as ldl

In [13]:
import scanpy.api as sc

### Load a dataset

In [6]:
h5_filename = data_path + 'raw_gene_bc_matrices_h5.h5'
metadata_filename = data_path + 'nsnp20.raw.sng.km_vb1_default.norm.meta.txt'

In [7]:
imp.reload(cs)
# Load the dataset
gene_dataset = cs.CropseqDataset(
    filename=h5_filename,
    metadata_filename=metadata_filename,
    batch='wells',
    use_labels='gene',
    save_path='')

Preprocessing CROP-seq dataset
Number of cells kept after filtering with metadata: 283634
Number of cells kept after removing all zero cells: 283634
Finished preprocessing CROP-seq dataset


### Read into scanpy

In [14]:
embedded_adata = sc.read('/Users/mincheolkim/Documents/nsnp20.raw.sng.km_vb1_default.pc60.norm.h5ad')

In [11]:
pd.Series(gene_dataset.ko_gene.reshape(-1)).value_counts()

NO_GUIDE    87336
FUBP1        4908
YEATS4       3863
IFI16        3403
VDR          2611
FUS          2604
GTF2H2       2520
NFATC3       2519
ZNF706       2438
RFX2         2431
DDB2         2385
ATF4         2343
ELK1         2343
EZH2         2317
ZNF146       2186
SMAD2        2172
RBBP7        2134
HDAC3        2118
NCOA3        2103
HOPX         2092
HMGB1        2049
ARID5A       2037
IRF2         2007
ZBED2        1969
GABPA        1964
IRF8         1906
SATB1        1900
MIER1        1824
ELF5         1817
PHB          1813
            ...  
SNW1          927
BATF          915
BACH2         911
ZNF207        904
PHTF2         861
JUNB          856
ZFP36L1       856
E2F4          850
ZNF593        848
MAZ           839
NFKBIA        828
NRF1          806
HIF1A         804
TAF9          798
RBPJ          793
STAT5A        773
SFPQ          758
POLR2A        756
STAT6         742
RUNX1         727
RPL7          707
NOC4L         627
PREB          594
JUN           587
SMARCE1   

### Create a dataframe with relevant cells

In [135]:
ko_genes = ['NO_GUIDE', 'BATF', 'IRF4', 'JUNB', 'STAT1']
genes_of_interest = [
    'BATF', 
    'IRF4', 
    'JUNB', 
    'RORC',
    'BCL6', 'MAF', 'IL10', 'IL17A', 'IL17B', 'CYCS', 'IL2', 'CMIP', 'IFNG', 'IL4', 'IL5', 'STAT1',
    'IL21', 'IL23R']#,'IL1RB1',]
goi_indices = [np.where(gene_dataset.gene_names == gene)[0][0] for gene in genes_of_interest]

In [136]:
gene_dataset.ko_gene_lookup

array(['ARID5A', 'ARID5B', 'ASCC1', 'ATF3', 'ATF4', 'ATF6', 'BACH1',
       'BACH2', 'BATF', 'BCLAF1', 'BHLHE40', 'CBFB', 'CEBPZ', 'CREM',
       'CTCF', 'CTCFL', 'DCP1A', 'DDB2', 'DDX3X', 'DNMT1', 'DPF2', 'DR1',
       'E2F4', 'EGR1', 'EGR2', 'ELF1', 'ELF5', 'ELK1', 'ELK4', 'ENO1',
       'ERG', 'ETS1', 'ETV1', 'EWSR1', 'EZH2', 'FLI1', 'FOSL2', 'FOXP1',
       'FUBP1', 'FUS', 'GABPA', 'GABPB1', 'GTF2H2', 'GTF2I', 'GTF3A',
       'HCFC1', 'HDAC3', 'HIF1A', 'HMGA1', 'HMGB1', 'HNRNPK', 'HOPX',
       'ID2', 'IFI16', 'IKZF1', 'IRF1', 'IRF2', 'IRF4', 'IRF8', 'JUN',
       'JUNB', 'KLF6', 'LRRFIP1', 'MAF1', 'MAFA', 'MAFK', 'MATR3', 'MAZ',
       'MIER1', 'MLX', 'MTA2', 'MYC', 'NCOA3', 'NCOA4', 'NFATC3',
       'NFKBIA', 'NOC4L', 'NONO', 'NO_GUIDE', 'NRF1', 'PARP1', 'PCBP1',
       'PCBP2', 'PHB', 'PHB2', 'PHF5A', 'PHTF2', 'PLAGL2', 'POLR2A',
       'PRDM1', 'PREB', 'RBBP7', 'RBPJ', 'RC3H1', 'RFX2', 'RPL7', 'RPL7A',
       'RUNX1', 'RUNX2', 'SATB1', 'SFPQ', 'SLC30A9', 'SMAD2', 'SMARCA5',
   

In [188]:
dfs=[]
for ko_gene in ko_genes:
    indices = (gene_dataset.labels == np.where(gene_dataset.ko_gene_lookup == ko_gene)[0][0]).reshape(-1)
    print(ko_gene, indices.sum())
    expr = gene_dataset.X[indices, :].astype(float)
    umi_counts = expr.sum(axis=1)
    df = pd.DataFrame(expr[:, goi_indices].todense()/umi_counts, columns=genes_of_interest)
    
    for gene in genes_of_interest:
        df[gene + '_std'] = (df[gene] - df[gene].mean())/np.sqrt(df[gene].var())
    df['ko_gene'] = ko_gene
    df['donor'] = pd.Series(gene_dataset.donor_batches[indices].reshape(-1)).astype(int)
    df['louvain'] = gene_dataset.louvain[indices].reshape(-1)
    df['well'] = gene_dataset.wells[indices].reshape(-1)
    dfs.append(df)
df = pd.get_dummies(pd.concat(dfs), columns=['ko_gene'])

NO_GUIDE 87336
BATF 915
IRF4 961
JUNB 856
STAT1 1458


### Compute covariances of interest and decide the group

In [189]:
def compute_point_cov(s1, s2):
   return df[s1 + '_std']*df[s2 + '_std']

In [190]:
groups = 'donor'

### Get some general sense of how sparse these genes are

IRF4 isnt detected for most cells, so I'm going to test BATF using IRF4 KO cells.

In [191]:
df['cov_BATF_RORC'] = compute_point_cov('BATF', 'RORC')
df['cov_IRF4_RORC'] = compute_point_cov('IRF4', 'RORC')
df['cov_BATF_IFNG'] = compute_point_cov('BATF', 'IFNG')
df['cov_IRF4_IFNG'] = compute_point_cov('IRF4', 'IFNG')

In [192]:
df['cov_BATF_RORC'] = compute_point_cov('BATF', 'RORC')
df['cov_IRF4_RORC'] = compute_point_cov('IRF4', 'RORC')
df['cov_STAT1_IFNG'] = compute_point_cov('STAT1', 'IFNG')
df['cov_IRF4_IFNG'] = compute_point_cov('IRF4', 'IFNG')

### Test differential BATF/RORC covariance between IRF4 KO cells and NO_GUIDE cells

In [194]:
print(smf.mixedlm(
    'cov_STAT1_IFNG ~ ko_gene_NO_GUIDE',
    df.query('ko_gene_STAT1 > 0 | ko_gene_NO_GUIDE > 0'),
    groups=df.query('ko_gene_STAT1 > 0 | ko_gene_NO_GUIDE > 0')[groups]).fit(maxiter=2000, method='nm').summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: cov_STAT1_IFNG
No. Observations: 88794   Method:             REML          
No. Groups:       9       Scale:              0.6086        
Min. group size:  7571    Likelihood:         -103952.9886  
Max. group size:  11927   Converged:          Yes           
Mean group size:  9866.0                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.037    0.020 -1.806 0.071 -0.077  0.003
ko_gene_NO_GUIDE   0.021    0.021  1.041 0.298 -0.019  0.062
Group Var          0.000    0.000                           



/anaconda3/envs/scvi/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### Scratch

In [23]:
temp = (gene_dataset.X > 0).sum(axis=0)

In [34]:
(temp > gene_dataset.X.shape[0]*0.05).sum()

4912

In [31]:
min(1, 300/400)

0.75

In [33]:
700/250000

0.0028